# 堆叠（Stacking）

堆叠是一种集成学习技术，它利用多个模型（例如决策树、KNN或SVM）的预测来构建一个新模型。这个新模型用于对测试集进行预测。

- **ML流水线回顾**：包括训练、测试和验证过程。

# 混合（Blending）

- 混合与堆叠非常相似，它同样使用基础模型提供的基础预测作为新特征，然后在这些新特征上训练一个新的元模型，以给出最终预测。唯一的区别在于，元模型的训练是在一个单独的保留集（例如训练数据的10%）上进行的，而不是在完整且折叠的训练集上进行。

# 套袋（Bagging）

- **动机**：如果在同一组数据上创建所有模型并将其组合，这会有用吗？由于这些模型接收到相同的输入，很有可能会得到相同的结果。那么我们该如何解决这个问题呢？一种技术是自助法（Bootstrapping）。

- **自助法**：是一种采样技术，我们从原始数据集中创建观察子集，并允许替换。子集的大小可以与原始集合相同，也可以不同。

- **套袋技术**（或称为自助聚合）使用这些子集（包）来获得完整集合的分布（complete set）的公平观念。为套袋创建的子集大小可能小于原始集合。

### 自助法（Bootstrap）

- **自助法**是一种统计技术，用于通过从原始数据中有替换地重新采样来估计统计量的抽样分布。这种方法由Bradley Efron在1979年引入。

- **操作步骤**：
  1. **原始数据集**：假设你有一个包含$n$个观测值的数据集$X = \{x_1, x_2, \cdots, x_n\}$。
  2. **重采样**：通过从原始数据集中随机抽取$n$个观测值（允许替换）来创建大量的自助样本。每个自助样本的大小与原始数据集相同，但由于替换采样，可能包含重复。
  3. **计算统计量**：对每个自助样本计算感兴趣的统计量（例如，均值、中位数、方差）。
  4. **估计分布**：使用一系列自助统计量来估计统计量的抽样分布。这可用于计算置信区间、标准误差和其他统计不确定性的度量。

- 从原始数据集创建多个子集，选择观测值时允许替换。
- 在这些子集上分别创建基础模型（弱模型）。这些模型并行运行，彼此独立。

通过这些高级集成技术，我们能够从不同的角度提升模型的性能和鲁棒性。堆叠和混合利用多个模型的预测来构建更强大的模型，而套袋和自助法通过在原始数据上进行有替换的重采样来创建多个子数据集，从而增强模型对数据的理解并降低过拟合的风险。

# 提升（Boosting）

提升是一种与套袋相反的技术，它不是并行地而是顺序地训练多个弱模型。每个模型都试图纠正前一个模型的错误，通过逐步增强来提高整体模型的准确性。

- **工作原理**：
  1. **开始**：首先训练一个基础模型，并在训练数据上进行预测。
  2. **增强**：接下来，增加被第一个模型错误预测的数据点的权重，然后基于更新后的数据训练第二个模型。
  3. **迭代**：这个过程重复进行，每次都增加之前模型错误预测的数据点的权重，直到达到预定的模型数量或模型性能不再显著提升为止。

- **优点**：提升技术特别擅长处理偏差较高的情况，因为它通过关注先前模型错分的样本来迭代改进。

- **挑战**：虽然提升能够减少偏差，但它可能增加模型的方差，因为它更加集中于训练数据，有过拟合的风险。

### 常见的提升算法

- **AdaBoost（自适应提升）**：通过调整每个数据点的权重，使得难以分类的数据点在后续的模型中获得更高的权重。
- **Gradient Boosting**：通过优化损失函数的梯度来不断改进模型，适用于各种损失函数的优化问题。
- **XGBoost、LightGBM 和 CatBoost**：这些是Gradient Boosting的高效实现，它们在处理大规模数据集时提供了更快的训练速度和更高的效率。

通过上述高级集成学习技术，我们可以构建更加强大和准确的预测模型，特别是在面对复杂的数据和任务时。堆叠、混合、套袋和提升各自有其优势和适用场景，正确选择和结合这些技术可以大大提升模型的性能。

# AdaBoost（自适应提升）

AdaBoost算法通过组合多个弱分类器来构建一个强分类器。以下是AdaBoost算法创建强分类器的详细步骤：

### 初始化

- 初始时，为所有训练样本分配等权重。如果有$N$个训练样本，每个样本的初始权重为$\frac{1}{N}$。

### 迭代训练

对于每次迭代$t$（从1到$T$，其中$T$是迭代总数）：

1. **训练弱分类器**：使用加权的训练样本训练一个弱分类器$h_t$。

2. **计算误差**：计算$h_t$的误差$\varepsilon_t$，误差是被错误分类样本的加权和，公式为：
   $$
   \varepsilon_t = \frac{\sum_{i=1}^{N} w_i I(y_i \neq h_t(x_i))}{\sum_{i=1}^{N} w_i}
   $$
   其中，$w_i$是第$i$个样本的权重，$y_i$是真实标签，$h_t(x_i)$是分类器$h_t$预测的标签，$I(·)$是指示函数，如果条件为真则返回1，否则返回0。

3. **计算分类器权重**：使用公式计算分类器的权重$\alpha_t$：
   $$
   \alpha_t = \frac{1}{2} \log\left(\frac{1 - \varepsilon_t}{\varepsilon_t}\right)
   $$
   误差$\varepsilon_t$越小，分类器权重$\alpha_t$越大。

4. **更新训练样本权重**：更新训练样本的权重：
   $$
   w_i \leftarrow w_i \exp(-\alpha_t y_i h_t(x_i))
   $$
   然后对权重进行归一化，使得它们的总和为1。

### 最终模型

- 经过$T$次迭代后，最终模型是弱分类器的加权组合：
  $$
  H(x) = \text{sign}\left(\sum \alpha_t h_t(x)\right)
  $$
  其中$\text{sign}(·)$是符号函数，对正输入返回+1，对负输入返回-1。

AdaBoost算法的核心在于它通过迭代地调整训练样本的权重，使得被先前弱分类器错误分类的样本在后续的训练中获得更高的权重。这样，弱分类器能够关注并更好地分类那些难以分类的样本，从而逐步提升整体模型的性能。


# Random Forest随机森林

随机森林（Random Forest）可以被比喻为机器学习中的一个群体决策团队。它结合了许多“树”（单个模型）的意见来做出更好的预测，从而创建出一个更加健壮和准确的整体模型。下面是随机森林算法的详细步骤：


#### 步骤1：选择数据和特征子集

- 在随机森林模型中，对于构建每一棵决策树，都会选择一部分数据点和一部分特征。简而言之，如果数据集有$k$条记录，那么就从中随机选取$n$条记录和$m$个特征。

#### 步骤2：构建个体决策树

- 对每个样本构建独立的决策树。

#### 步骤3：每棵决策树生成输出

- 每棵决策树都会给出一个预测结果。

#### 步骤4：基于投票或平均值确定最终输出

- 对于**分类Classification任务**，最终输出是基于**多数投票**（Majority Voting）确定的。也就是说，最终的预测结果是由多数决策树支持的结果。
- 对于**回归Regression任务**，最终输出是通过对各个决策树输出的**平均值**（Averaging）来确定的。

### 随机森林的优点

1. **准确性**：通过结合多个决策树的预测，随机森林通常能够提供高准确率的模型。
2. **鲁棒性**：由于采用了多棵树，即使其中一些树的表现不佳，整体模型仍然能够保持较好的性能。
3. **抗过拟合**：通过在构建决策树时引入随机性，随机森林能够有效降低过拟合的风险。
4. **处理高维数据**：随机森林能够处理大量特征而不需要进行特征选择。

### 应用场景

随机森林在众多领域都有应用，包括但不限于金融风险评估、医疗诊断、股市分析和生态模型预测等。由于其高效性和准确性，随机森林成为了许多数据科学家和研究人员常用的机器学习算法之一。